In [3]:
# STEP 1: Install dependencies
!pip install pymongo[srv] pandas dnspython

# STEP 2: Connect to MongoDB (Use MongoDB Atlas or Localhost URI)
from pymongo import MongoClient
import pandas as pd

# Replace with your MongoDB Atlas connection string or local URI
MONGO_URI = "mongodb+srv://<username>:<password>@cluster0.mongodb.net/?retryWrites=true&w=majority"
client = MongoClient(MONGO_URI)
db = client["property_db"]
properties = db["properties"]

# STEP 3: Load CSV file into MongoDB
csv_path = "/mnt/data/db424fd9fb74_1748258398689.csv"
df = pd.read_csv(csv_path)
df.fillna("", inplace=True)  # Handle missing values

# Clear old data (for rerun)
properties.delete_many({})

# Insert to MongoDB
records = df.to_dict(orient='records')
properties.insert_many(records)
print(f"✅ Imported {len(records)} records into MongoDB")

# STEP 4: CRUD OPERATIONS
# CREATE
def create_property(data, created_by):
    data['createdBy'] = created_by
    result = properties.insert_one(data)
    return str(result.inserted_id)

# READ (All or Filtered)
def get_properties(filters={}):
    return list(properties.find(filters, {'_id': 0}))

# UPDATE
def update_property(property_id, updated_data, user_id):
    result = properties.update_one({'_id': property_id, 'createdBy': user_id}, {'$set': updated_data})
    return result.modified_count

# DELETE
def delete_property(property_id, user_id):
    result = properties.delete_one({'_id': property_id, 'createdBy': user_id})
    return result.deleted_count

# STEP 5: ADVANCED FILTERING
# Example: Search properties with price < 500000 and city = "Mumbai"
filters = {'price': {"$lt": 500000}, 'city': "Mumbai"}
matched_props = get_properties(filters)
print(f"🔍 Found {len(matched_props)} properties under 5L in Mumbai")

# STEP 6: USER SYSTEM (Mocked for Colab)
users = db["users"]
favorites = db["favorites"]
recommendations = db["recommendations"]

# Register user
def register_user(email, password):
    if users.find_one({'email': email}):
        return "User already exists."
    users.insert_one({'email': email, 'password': password})
    return "✅ Registered"

# Login
def login(email, password):
    user = users.find_one({'email': email, 'password': password})
    return user if user else "❌ Invalid credentials"

# STEP 7: FAVORITES CRUD
def favorite_property(user_email, property_id):
    favorites.insert_one({'email': user_email, 'property_id': property_id})

def get_user_favorites(user_email):
    return list(favorites.find({'email': user_email}, {'_id': 0}))

# STEP 8: RECOMMEND PROPERTY TO ANOTHER USER
def recommend_property(from_email, to_email, property_id):
    if not users.find_one({'email': to_email}):
        return "Recipient does not exist"
    recommendations.insert_one({'from': from_email, 'to': to_email, 'property_id': property_id})
    return "✅ Property recommended"

def get_recommendations_received(email):
    return list(recommendations.find({'to': email}, {'_id': 0}))

# ✅ TESTING
print(register_user("alice@example.com", "1234"))
print(register_user("bob@example.com", "abcd"))
print(login("bob@example.com", "abcd"))

favorite_property("bob@example.com", "property_123")
print(get_user_favorites("bob@example.com"))

recommend_property("alice@example.com", "bob@example.com", "property_456")
print(get_recommendations_received("bob@example.com"))


ConfigurationError: The DNS query name does not exist: _mongodb._tcp.cluster0.mongodb.net.